In [1]:
import os
import re
import time
import pickle
import itertools
from math import pi

import numpy as np
import pandas as pd

from pyomo.environ import *

import matplotlib.pyplot as plt
np.random.seed(10)

## Paths

In [2]:
class DirectoryPaths(object):
    
    def __init__(self):
        self.data_dir = os.path.join(os.path.curdir, os.path.pardir, os.path.pardir, 'data')
        self.output_dir = os.path.join(os.path.curdir, 'output')

paths = DirectoryPaths()

## Model data

In [3]:
class RawData(object):
    
    def __init__(self):
        
        # Paths to directories
        DirectoryPaths.__init__(self)
        
        
        # Network data
        # ------------
        # Nodes
        self.df_n = pd.read_csv(os.path.join(self.data_dir, 'network_nodes.csv'), index_col='NODE_ID')

        # AC edges
        self.df_e = pd.read_csv(os.path.join(self.data_dir, 'network_edges.csv'), index_col='LINE_ID')

        # HVDC links
        self.df_hvdc_links = pd.read_csv(os.path.join(self.data_dir, 'network_hvdc_links.csv'), index_col='HVDC_LINK_ID')

        # AC interconnector links
        self.df_ac_i_links = pd.read_csv(os.path.join(self.data_dir, 'network_ac_interconnector_links.csv'), index_col='INTERCONNECTOR_ID')

        # AC interconnector flow limits
        self.df_ac_i_limits = pd.read_csv(os.path.join(self.data_dir, 'network_ac_interconnector_flow_limits.csv'), index_col='INTERCONNECTOR_ID')


        # Generators
        # ----------       
        # Generating unit information
        self.df_g = pd.read_csv(os.path.join(self.data_dir, 'generators.csv'), index_col='DUID', dtype={'NODE': int})
        self.df_g['SRMC_2016-17'] = self.df_g['SRMC_2016-17'].map(lambda x: x + np.random.uniform(0, 2))
        
        # Station owners
        self.df_g_own = (pd.read_csv(os.path.join(self.data_dir, 'PUBLIC_DVD_STATIONOWNER_201706010000.CSV'), 
                                    skiprows=1, skipfooter=1, engine='python', parse_dates=['LASTCHANGED'])
                         .sort_values('LASTCHANGED', ascending=False)
                         .drop_duplicates('STATIONID', keep='first'))

        
        # Signals
        # -------
        with open(os.path.join(os.path.curdir, os.path.pardir, '1_operating_scenarios', 'output', 'weekly_scenarios.pickle'), 'rb') as f:
            self.df_scenarios = pickle.load(f)

# Create object containing raw model data
raw_data = RawData() 

## Organise model data

In [4]:
class OrganiseData(object):
    "Organise data to be used in mathematical program"
    
    def __init__(self):
        # Load model data
        RawData.__init__(self)
        
        
        def add_participantids_to_generator_dataframe(self):
            "Add station owner IDs to generators DataFrame"
            
            # New generators DataFrame - merging Participant IDs using Station IDs
            df_g = (self.df_g.reset_index()
                    .merge(self.df_g_own[['PARTICIPANTID', 'STATIONID']], left_on='STATIONID', right_on='STATIONID')
                    .set_index('DUID'))
            return df_g
        
        self.df_g = add_participantids_to_generator_dataframe(self)

        

    def get_admittance_matrix(self):
        "Construct admittance matrix for network"

        # Initialise dataframe
        df_Y = pd.DataFrame(data=0j, index=self.df_n.index, columns=self.df_n.index)

        # Off-diagonal elements
        for index, row in self.df_e.iterrows():
            fn, tn = row['FROM_NODE'], row['TO_NODE']
            df_Y.loc[fn, tn] += - (1 / (row['R_PU'] + 1j * row['X_PU'])) * row['NUM_LINES']
            df_Y.loc[tn, fn] += - (1 / (row['R_PU'] + 1j * row['X_PU'])) * row['NUM_LINES']

        # Diagonal elements
        for i in self.df_n.index:
            df_Y.loc[i, i] = - df_Y.loc[i, :].sum()

        # Add shunt susceptance to diagonal elements
        for index, row in self.df_e.iterrows():
            fn, tn = row['FROM_NODE'], row['TO_NODE']
            df_Y.loc[fn, fn] += (row['B_PU'] / 2) * row['NUM_LINES']
            df_Y.loc[tn, tn] += (row['B_PU'] / 2) * row['NUM_LINES']

        return df_Y
    
    
    def get_HVDC_incidence_matrix(self):
        "Incidence matrix for HVDC links"
        
        # Incidence matrix for HVDC links
        df = pd.DataFrame(index=self.df_n.index, columns=self.df_hvdc_links.index, data=0)

        for index, row in self.df_hvdc_links.iterrows():
            # From nodes assigned a value of 1
            df.loc[row['FROM_NODE'], index] = 1

            # To nodes assigned a value of -1
            df.loc[row['TO_NODE'], index] = -1
        
        return df
    
    
    def get_all_ac_edges(self):
        "Tuples defining from and to nodes for all AC edges (forward and reverse)"
        
        # Set of all AC edges
        edge_set = set()
        
        # Loop through edges, add forward and reverse direction indice tuples to set
        for index, row in model_data.df_e.iterrows():
            edge_set.add((row['FROM_NODE'], row['TO_NODE']))
            edge_set.add((row['TO_NODE'], row['FROM_NODE']))
        
        return edge_set
    
    def get_network_graph(self):
        "Graph containing connections between all network nodes"
        network_graph = {n: set() for n in model_data.df_n.index}

        for index, row in model_data.df_e.iterrows():
            network_graph[row['FROM_NODE']].add(row['TO_NODE'])
            network_graph[row['TO_NODE']].add(row['FROM_NODE'])
        
        return network_graph
    
    
    def get_all_dispatchable_fossil_generator_duids(self):
        "Fossil dispatch generator DUIDs"
        
        # Filter - keeping only fossil and scheduled generators
        mask = (model_data.df_g['FUEL_CAT'] == 'Fossil') & (model_data.df_g['SCHEDULE_TYPE'] == 'SCHEDULED')
        
        return model_data.df_g[mask].index    
    
    
    def get_intermittent_dispatch(self):
        "Dispatch from intermittent generators (solar, wind)"
        
        # Intermittent generator DUIDs
        intermittent_duids_mask = model_data.df_g['FUEL_CAT'].isin(['Wind', 'Solar'])
        intermittent_duids = model_data.df_g.loc[intermittent_duids_mask].index

        # Intermittent dispatch aggregated by node
        intermittent_dispatch =(model_data.df_dispatch.reindex(columns=intermittent_duids, fill_value=0)
                                .T
                                .join(model_data.df_g[['NODE']])
                                .groupby('NODE').sum()
                                .reindex(index=model_data.df_n.index, fill_value=0)
                                .T)
        
        # Make sure columns are of type datetime
        intermittent_dispatch.index = intermittent_dispatch.index.astype('datetime64[ns]')
        
        return intermittent_dispatch
    
    
    def get_hydro_dispatch(self):
        "Dispatch from hydro plant"
        
        # Dispatch from hydro plant
        hydro_duids_mask = self.df_g['FUEL_CAT'].isin(['Hydro'])
        hydro_duids = self.df_g.loc[hydro_duids_mask].index

        # Hydro plant dispatch aggregated by node
        hydro_dispatch = (self.df_dispatch.reindex(columns=hydro_duids, fill_value=0)
                          .T
                          .join(model_data.df_g[['NODE']])
                          .groupby('NODE').sum()
                          .reindex(index=self.df_n.index, fill_value=0)
                          .T)
        
        # Make sure columns are of type datetime
        hydro_dispatch.index = hydro_dispatch.index.astype('datetime64[ns]')
        
        return hydro_dispatch
    
    
    def get_reference_nodes(self):
        "Get reference node IDs"
        
        # Filter Regional Reference Nodes (RRNs) in Tasmania and Victoria.
        mask = (model_data.df_n['RRN'] == 1) & (model_data.df_n['NEM_REGION'].isin(['TAS1', 'VIC1']))
        reference_node_ids = model_data.df_n[mask].index
        
        return reference_node_ids
    
    
    def get_node_demand(self):   
        "Compute demand at each node for a given time period, t"

        def _node_demand(row):
            # NEM region for a given node
            region = row['NEM_REGION']

            # Load at node
            demand = self.df_load.loc[:, region] * row['PROP_REG_D']

            return demand
        node_demand = self.df_n.apply(_node_demand, axis=1).T
        
        return node_demand
    
    def get_generator_node_map(self, generators):
        "Get set of generators connected to each node"
        generator_node_map = (self.df_g.reindex(index=generators)
                              .reset_index()
                              .rename(columns={'OMEGA_G': 'DUID'})
                              .groupby('NODE').agg(lambda x: set(x))['DUID']
                              .reindex(self.df_n.index, fill_value=set()))
        
        return generator_node_map
    
    def get_ac_interconnector_summary(self):
        "Summarise aggregate flow limit information for AC interconnectors"

        # Check that from and to regions conform with regional power flow limit directions
        def check_flow_direction(row):
            if (row['FROM_REGION'] == self.df_ac_i_limits.loc[row.name, 'FROM_REGION']) & (row['TO_REGION'] == model_data.df_ac_i_limits.loc[row.name, 'TO_REGION']):
                return True
            else:
                return False
        # Flow directions are consistent between link and limit DataFrames if True
        flow_directions_conform = self.df_ac_i_links.apply(check_flow_direction, axis=1).all()
        if flow_directions_conform:
            print('Flow directions conform with regional flow limit directions: {0}'.format(flow_directions_conform))
        else:
            raise(Exception('Link flow directions inconsitent with regional flow forward limit definition'))

        # Forward limit
        df_forward = self.df_ac_i_links.apply(lambda x: (x['FROM_NODE'], x['TO_NODE']), axis=1).reset_index().groupby('INTERCONNECTOR_ID').agg(lambda x: list(x)).join(model_data.df_ac_i_limits['FORWARD_LIMIT_MW'], how='left').rename(columns={0: 'branches', 'FORWARD_LIMIT_MW': 'limit'})
        df_forward['new_index'] = df_forward.apply(lambda x: x.name + '-FORWARD', axis=1)
        df_forward.set_index('new_index', inplace=True)

        # Reverse limit
        df_reverse = self.df_ac_i_links.apply(lambda x: (x['TO_NODE'], x['FROM_NODE']), axis=1).reset_index().groupby('INTERCONNECTOR_ID').agg(lambda x: list(x)).join(model_data.df_ac_i_limits['REVERSE_LIMIT_MW'], how='left').rename(columns={0: 'branches', 'REVERSE_LIMIT_MW': 'limit'})
        df_reverse['new_index'] = df_reverse.apply(lambda x: x.name + '-REVERSE', axis=1)
        df_reverse.set_index('new_index', inplace=True)
        df_ac_limits = pd.concat([df_forward, df_reverse])

        return df_ac_limits

# Create object containing organised model data
model_data = OrganiseData()

## Model

In [27]:
def create_model(use_pu=False):
    "Create model object"
    
    # Initialise model
    model = ConcreteModel()

    # Sets
    # ----   
    # Nodes
    model.OMEGA_N = Set(initialize=model_data.df_n.index)

    # Generators
    model.OMEGA_G = Set(initialize=model_data.get_all_dispatchable_fossil_generator_duids())

    # AC edges
    ac_edges = model_data.get_all_ac_edges()
    model.OMEGA_NM = Set(initialize=ac_edges)
    
    # Sets of branches for which aggregate AC interconnector limits are defined
    ac_limits = model_data.get_ac_interconnector_summary()
    model.OMEGA_J = Set(initialize=ac_limits.index)
    
    # HVDC links
    model.OMEGA_H = Set(initialize=model_data.df_hvdc_links.index)


    # Parameters
    # ----------
    # System base power
    model.BASE_POWER = Param(initialize=100)
    
    # Emissions intensity baseline
    model.PHI = Param(initialize=0, mutable=True)
    
    # Permit price
    model.TAU = Param(initialize=0, mutable=True)
    
    # Generator emissions intensities
    def E_RULE(model, g):
        return float(model_data.df_g.loc[g, 'EMISSIONS'])
    model.E = Param(model.OMEGA_G, rule=E_RULE)
    
    # Admittance matrix
    admittance_matrix = model_data.get_admittance_matrix()
    def B_RULE(model, n, m):
        if use_pu:
            return float(np.imag(admittance_matrix.loc[n, m]))
        else:
            return model.BASE_POWER * float(np.imag(admittance_matrix.loc[n, m]))
    model.B = Param(model.OMEGA_NM, rule=B_RULE)

    # Reference nodes
    reference_nodes = model_data.get_reference_nodes()
    def S_RULE(model, n):
        if n in reference_nodes:
            return 1
        else:
            return 0
    model.S = Param(model.OMEGA_N, rule=S_RULE)

    # Generator short-run marginal costs
    def C_RULE(model, g):
        marginal_cost = float(model_data.df_g.loc[g, 'SRMC_2016-17'])
        if use_pu:
            return marginal_cost / model.BASE_POWER
        else:
            return marginal_cost
    model.C = Param(model.OMEGA_G, rule=C_RULE)

    # Demand
    model.D = Param(model.OMEGA_N, initialize=0, mutable=True)
    
    # Max voltage angle difference between connected nodes
    model.THETA_DELTA = Param(initialize=float(pi / 2))
    
    # HVDC incidence matrix
    hvdc_incidence_matrix = model_data.get_HVDC_incidence_matrix()
    def K_RULE(model, n, h):
        return float(hvdc_incidence_matrix.loc[n, h])
    model.K = Param(model.OMEGA_N, model.OMEGA_H, rule=K_RULE)    
    
    # Aggregate AC interconnector flow limits
    def F_RULE(model, j):
        power_flow_limit = float(ac_limits.loc[j, 'limit'])
        if use_pu:
            return power_flow_limit / model.BASE_POWER
        else:
            return power_flow_limit
    model.F = Param(model.OMEGA_J, rule=F_RULE)
    
    # Fixed power injections
    model.R = Param(model.OMEGA_N, initialize=0, mutable=True)
    
    
    # Variables
    # ---------
    # Generator output
    def P_RULE(model, g):
        registered_capacity = float(model_data.df_g.loc[g, 'REG_CAP'])
        if use_pu:
            return (0, registered_capacity / model.BASE_POWER)
        else:
            return (0, registered_capacity)
    model.p = Var(model.OMEGA_G, bounds=P_RULE)

    # HVDC flows
    def P_H_RULE(model, h):
        forward_flow_limit = float(model_data.df_hvdc_links.loc[h, 'FORWARD_LIMIT_MW'])
        reverse_flow_limit = float(model_data.df_hvdc_links.loc[h, 'REVERSE_LIMIT_MW'])
        if use_pu:
            return (- reverse_flow_limit / model.BASE_POWER, forward_flow_limit / model.BASE_POWER)
        else:
            return (- reverse_flow_limit, forward_flow_limit)
    model.p_H = Var(model.OMEGA_H, bounds=P_H_RULE)
    
    # Node voltage angles
    model.theta = Var(model.OMEGA_N)


    # Constraints
    # -----------
    # Power balance
    generator_node_map = model_data.get_generator_node_map(model.OMEGA_G)
    network_graph = model_data.get_network_graph()
    def POWER_BALANCE_RULE(model, n):
        return (model.D[n] 
                - model.R[n]
                - sum(model.p[g] for g in generator_node_map[n]) 
                + sum(model.B[n, m] * (model.theta[n] - model.theta[m]) for m in network_graph[n]) 
                + sum(model.K[n, h] * model.p_H[h] for h in model.OMEGA_H) == 0)
    model.POWER_BALANCE = Constraint(model.OMEGA_N, rule=POWER_BALANCE_RULE)

    # Reference angle
    def REFERENCE_ANGLE_RULE(model, n):
        if model.S[n] == 1:
            return model.theta[n] == 0
        else:
            return Constraint.Skip
    model.REFERENCE_ANGLE = Constraint(model.OMEGA_N, rule=REFERENCE_ANGLE_RULE)

    # Voltage angle difference constraint
    def VOLTAGE_ANGLE_DIFFERENCE_RULE(model, n, m):
        return model.theta[n] - model.theta[m] - model.THETA_DELTA <= 0
    model.VOLTAGE_ANGLE_DIFFERENCE = Constraint(model.OMEGA_NM, rule=VOLTAGE_ANGLE_DIFFERENCE_RULE)
    
    # AC interconnector flow constraints
    def AC_FLOW_RULE(model, j):
        return sum(model.B[n, m] * (model.theta[n] - model.theta[m]) for n, m in ac_limits.loc[j, 'branches'])
    model.AC_FLOW = Expression(model.OMEGA_J, rule=AC_FLOW_RULE)
    
    def AC_POWER_FLOW_LIMIT_RULE(model, j):
        return model.AC_FLOW[j] - model.F[j] <= 0
    model.AC_POWER_FLOW_LIMIT = Constraint(model.OMEGA_J, rule=AC_POWER_FLOW_LIMIT_RULE)
    

    # Objective
    # ---------
    model.x_1 = Var(model.OMEGA_J, within=NonNegativeReals)
    model.x_2 = Var(model.OMEGA_J, within=NonNegativeReals)
    
    def ABS_HVDC_FLOW_1_RULE(model, j):
        return model.x_1[j] >= model.AC_FLOW[j]
    model.ABS_HVDC_FLOW_1 = Constraint(model.OMEGA_J, rule=ABS_HVDC_FLOW_1_RULE)
    
    def ABS_HVDC_FLOW_2_RULE(model, j):
        return model.x_2[j] >= - model.AC_FLOW[j]
    model.ABS_HVDC_FLOW_2 = Constraint(model.OMEGA_J, rule=ABS_HVDC_FLOW_2_RULE)
    
    def HVDC_FLOW_COST_RULE(model):
        if use_pu:
            return 10 / model.BASE_POWER
        else:
            return 10
    model.HVDC_FLOW_COST = Param(initialize=HVDC_FLOW_COST_RULE)
    
    model.OBJECTIVE = Objective(expr=sum((model.C[g] + ((model.E[g] - model.PHI) * model.TAU)) * model.p[g] for g in model.OMEGA_G) + sum((model.x_1[j] + model.x_2[j]) * model.HVDC_FLOW_COST for j in model.OMEGA_J))

    return model

Initialise model object

In [28]:
model = create_model(use_pu=True)

Flow directions conform with regional flow limit directions: True


Setup solve

In [29]:
# Setup solver
# ------------
solver = 'gurobi'
solver_io = 'lp'
model.dual = Suffix(direction=Suffix.IMPORT)
opt = SolverFactory(solver, solver_io=solver_io)

Update parameters and solve model

In [32]:
def solve_scenario(model, permit_price, baseline, use_pu, week, scenario):
    "Update demand and fixed power injection parameters and solve model"
    
    # Update permit price
    if use_pu:
        model.TAU = permit_price / model.BASE_POWER.value
    else:
        model.TAU = permit_price
    
    # Update emissions intensity baseline
    model.PHI = baseline
    
    # Update parameters for each node
    for n in model.OMEGA_N:
        # Node demand
        node_demand = model_data.df_scenarios.loc[(n, 'demand'), (week, scenario)]

        # Hydro power injection
        node_hydro = model_data.df_scenarios.loc[(n, 'hydro'), (week, scenario)]

        # Intermittent power injection
        node_intermittent = model_data.df_scenarios.loc[(n, 'intermittent'), (week, scenario)]

        # If using per-unit scaling
        if use_pu:
            model.D[n] = node_demand / model.BASE_POWER.value
            model.R[n] = (node_hydro + node_intermittent) / model.BASE_POWER.value
        else:
            model.D[n] = node_demand
            model.R[n] = node_hydro + node_intermittent
    
    # Solve model
    res = opt.solve(model, keepfiles=False, tee=True, warmstart=False)
    model.solutions.store_to(res)

    # Place results in DataFrame
    df = pd.DataFrame(res['Solution'][0])
    
    return df

use_pu = True
week = 1
scenario = 1
solve_scenario(model=model, permit_price=40, baseline=0.8, use_pu=True, week=1, scenario=1)

Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 3361 rows, 1066 columns and 8421 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 8e+04]
  Objective range [1e-01, 2e+00]
  Bounds range    [2e-01, 7e+00]
  RHS range       [1e-05, 2e+01]
Presolve removed 2663 rows and 681 columns
Presolve time: 0.02s
Presolved: 698 rows, 833 columns, 2752 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     506    5.4784630e+01   0.000000e+00   0.000000e+00      0s

Solved in 506 iterations and 0.03 seconds
Optimal objective  5.478463039e+01


,Gap,Status,Message,Problem,Objective,Variable,Constraint
ABS_HVDC_FLOW_1[N-Q-MNSP1-FORWARD],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}
ABS_HVDC_FLOW_1[N-Q-MNSP1-REVERSE],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.048756680073}
ABS_HVDC_FLOW_1[NSW1-QLD1-FORWARD],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}
ABS_HVDC_FLOW_1[NSW1-QLD1-REVERSE],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}
ABS_HVDC_FLOW_1[V-SA-FORWARD],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.0124856551231}
ABS_HVDC_FLOW_1[V-SA-REVERSE],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': 0.0}
ABS_HVDC_FLOW_1[VIC1-NSW1-FORWARD],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': 0.0}
ABS_HVDC_FLOW_1[VIC1-NSW1-REVERSE],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}
ABS_HVDC_FLOW_2[N-Q-MNSP1-FORWARD],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}
ABS_HVDC_FLOW_2[N-Q-MNSP1-REVERSE],0.0,optimal,Model was solved to optimality (subject to tol...,NaN,NaN,NaN,{'Dual': -0.1}


Per-unit scaled model

No scaling 

In [ ]:
model_ab, df_ab = run_model(use_pu=False)

In [ ]:
df_ab

In [ ]:
df_pu

In [ ]:
def compute_variable_difference(set_contains):
    "Compare variable output from scaled and non-scaled models"
    
    mask_pu = df_pu.index.str.startswith(set_contains)
    pu = df_pu.loc[mask_pu, 'Variable'].map(lambda x: x['Value'])

    mask_ab = df_ab.index.str.startswith(set_contains)
    ab = df_ab.loc[mask_ab, 'Variable'].map(lambda x: x['Value'])
    
    if set_contains == 'theta[':
        diff = ab - pu
    else:
        diff = ab - (pu * 100)
    
    return diff.abs().max()
print('Max absolute power output difference between models:', compute_variable_difference('p['))
print('Max absolute voltage angle difference between models:', compute_variable_difference('theta['))
print('Max HVDC flow difference:', compute_variable_difference('p_H['))

In [ ]:
model_pu.p.display()